In [1]:
!pip install pymystem3
!pip install dostoevsky
!pip install pymorphy2

In [17]:
import pandas as pd
import numpy as np
import nltk         #  pip install nltk
import pymorphy2    #  pip install pymorphy2 pymorphy2-dicts
nltk.download('punkt')
from pymystem3 import Mystem
from datetime import datetime
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\msvic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
dfcolumns = pd.read_csv(r'C:/Users/msvic/ВКР/Kalininsky_comments.csv',
                        nrows = 1, delimiter=';', encoding = 'UTF-8')
df = pd.read_csv(r'C:/Users/msvic/ВКР/Kalininsky_comments.csv',
                  header = None,
                  delimiter=';',
                  skiprows = 1,
                  encoding='UTF-8',
                  usecols = list(range(len(dfcolumns.columns))),
                  names = dfcolumns.columns)

In [19]:
length = len(df)
print(length)

260737


In [20]:
df.head()[:1]

,ID,ID автора,Ссылка на автора,Имя и фамилия автора,Пол автора,Ссылка на комментарий,Номер сообщества,Дата и время,Время,Обращение,Текст комментария,Число лайков к комментарию,Длина
0,1,601628352,https://vk.com/id601628352,Михаил Лахин,М,https://vk.com/wall-23904_314769?reply=314771,NaN,01.05.2023 22:34,22:34,0,Народ и партия едины,0.0,20


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260737 entries, 0 to 260736
Data columns (total 13 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ID                          260737 non-null  int64  
 1   ID автора                   260737 non-null  int64  
 2   Ссылка на автора            260737 non-null  object 
 3   Имя и фамилия автора        260172 non-null  object 
 4   Пол автора                  250718 non-null  object 
 5   Ссылка на комментарий       260737 non-null  object 
 6   Номер сообщества            0 non-null       float64
 7   Дата и время                260737 non-null  object 
 8   Время                       260737 non-null  object 
 9   Обращение                   260737 non-null  int64  
 10  Текст комментария           254775 non-null  object 
 11  Число лайков к комментарию  260735 non-null  float64
 12  Длина                       260737 non-null  int64  
dtypes: float64(2),

In [22]:
cols = [col for col in df.columns]
print(cols)

['ID', 'ID автора', 'Ссылка на автора', 'Имя и фамилия автора', 'Пол автора', 'Ссылка на комментарий', 'Номер сообщества', 'Дата и время', 'Время', 'Обращение', 'Текст комментария', 'Число лайков к комментарию', 'Длина']


In [23]:
import pandas as pd
import re

# Определяем функцию для извлечения цифр из строки
def extract_numbers(s):
    return re.findall(r'-(\d+)_', s)[0]

# Создаем новый столбец с извлеченными цифрами
df['ССЫЛКА НА СООБЩЕСТВО'] = df['Ссылка на комментарий'].apply(lambda s: 'https://vk.com/club' + str(extract_numbers(s)))

In [25]:
df['ТЕКСТ2'] = df['Текст комментария']

In [26]:
import re

print("Cleaning text...")
with open("C:/myR/stopwords.txt", "r", encoding="UTF-8") as file:
    stop_words = set(map(str.strip, file))


df['ТЕКСТ2'] = df['ТЕКСТ2'].str.lower().str.replace("<[^>]*>", " ", regex=True) #Удаляем знаки
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("\r|\n", "", regex=True) #Удаляем переходы на новую строку
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("\[[^[]*\|", "", regex=True)
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("[^\w\s]", " ", regex=True)
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("ё", "е", regex=True) #Заменяем "ё" на "е"
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("\s\s+", " ", regex=True)
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("\d", "", regex=True) #Удаляем цифры
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("  ", " ", regex=True) #Удаляем двойные пробелы
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.strip() #Удаляем пробелы в обоих концах строк
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("[a-z]", "", regex=True)
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("[A-Z]", "", regex=True)


# Удаляем стоп-слова
df['ТЕКСТ2'] = df['ТЕКСТ2'].apply(lambda x: ' '.join(filter(lambda word: word not in stop_words, x.split())) if isinstance(x, str) else x)

print("Successful!")
print("---------------------------------")


Cleaning text...
Successful!
---------------------------------


In [27]:
print(df['ТЕКСТ2'])

0                                        народ партия едины
1         наверное первомайские демонстрации ходил колле...
2         года ходил городе постоянно протяжении лет про...
3                           спасибо публикуйте таких статей
4                                          рахова поздравит
                                ...                        
260732    наши источники мои утверждения несколько публи...
260733    добрый чему вопросы такого характера стали гор...
260734            адрес никто просит понимать вопрос живете
260735    олечка несмотря вашу явную нелюбовь собакам лю...
260736    животные кошки собаки взятые приюта троллейбус...
Name: ТЕКСТ2, Length: 260737, dtype: object

In [29]:
import pandas as pd
import openpyxl

# читаем файл Excel
wb = openpyxl.load_workbook('C:/Users/msvic/ВКР/Base.xlsx')
ws = wb.active


# получаем значения столбцов H, C и D - можно через ord, например: ord('C')-64
links = [cell.value for row in ws.iter_rows(min_row=2, max_row=56, min_col=8, max_col=8) for cell in row] # H - 8, строки с 2 по 56
types = [cell.value for row in ws.iter_rows(min_row=2, max_row=56, min_col=3, max_col=3) for cell in row]
subtypes = [cell.value for row in ws.iter_rows(min_row=2, max_row=56, min_col=4, max_col=4) for cell in row]

# присваиваем значения столбца 'type' в зависимости от значения столбца 'link'
for i, row in df.iterrows():
    if row['ССЫЛКА НА СООБЩЕСТВО'] in links:
        index = links.index(row['ССЫЛКА НА СООБЩЕСТВО'])
        df.at[i, 'ТИП'] = types[index]
        df.at[i, 'ПОДТИП'] = subtypes[index]

In [30]:
print(df.head())

,ID,ID автора,Ссылка на автора,Имя и фамилия автора,Пол автора,Ссылка на комментарий,Номер сообщества,Дата и время,Время,Обращение,Текст комментария,Число лайков к комментарию,Длина,ССЫЛКА НА СООБЩЕСТВО,ТЕКСТ2,ТИП,ПОДТИП
0,1,601628352,https://vk.com/id601628352,Михаил Лахин,М,https://vk.com/wall-23904_314769?reply=314771,NaN,01.05.2023 22:34,22:34,0,Народ и партия едины,0.0,20,https://vk.com/club23904,народ партия едины,official,dist
1,2,13329321,https://vk.com/id13329321,Анна Коробкова,Ж,https://vk.com/wall-23904_314758?reply=314759,NaN,01.05.2023 18:06,18:06,0,"Наверное, он и на первомайские демонстрации хо...",0.0,149,https://vk.com/club23904,наверное первомайские демонстрации ходил колле...,official,dist
2,3,1347813,https://vk.com/id1347813,Андрей Ходырев,М,https://vk.com/wall-23904_314758?reply=314773,NaN,01.05.2023 23:12,23:12,2,"[id13329321|Анна], два года не ходил - не был ...",0.0,116,https://vk.com/club23904,года ходил городе постоянно протяжении лет про...,official,dist
3,4,144899983,https://vk.com/id144899983,Александр Пенин,М,https://vk.com/wall-23904_314758?reply=314772,NaN,01.05.2023 22:48,22:48,0,"Спасибо, публикуйте больше таких статей.",1.0,40,https://vk.com/club23904,спасибо публикуйте таких статей,official,dist
4,5,3648052,https://vk.com/id3648052,Виталий Селиванов,М,https://vk.com/wall-23904_314714?reply=314717,NaN,01.05.2023 9:16,9:16,0,А Рахова поздравит?,1.0,19,https://vk.com/club23904,рахова поздравит,official,dist


In [31]:
df['ТЕКСТ3'] = df['ТЕКСТ2']

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from pymystem3 import Mystem

### Здесь мы создаем функцию lemmatize_text, которая принимает на вход текст и возвращает лемматизированный текст 

def lemmatize_text(text):                         
    m = Mystem()
    lemmas = m.lemmatize(text)                       #лемматизируем текст
    list_len = len(lemmas)                           #узнаем количество слов в list'е lemmas (удвоенное)
    if list_len > 0:                                 #проверяем есть ли слова в посте
        if list_len == 2:                            #проверяем больше ли одного слова в посте
            mystr = lemmas[0]                        #приравниваем строку для записи в csv файл с единственным словом в посте
        elif list_len > 2:                           # == если в посте больше двух слов
            list_len -= 2                            # кол-во элементов в списке на 1 больше индекса последнего элемента списка;
            mystr = ""                               # последний элемент данного списка == '\n'
            i = 0
            while i <= list_len:                       #цикл считывания лемм
                mystr += lemmas[i] + " "               #прибавляем слова к строке для записи с пробелом на конце
                i += 2                               #так как в списке все слова имеют четный индекс, то прибавляем 2
        return mystr.strip()
    else:
        return ""                               


    
def process_chunk(chunk):                             # каждый chunk состояит из 1/n куска df
    
    for i, row in chunk.iterrows():
        text = row['ТЕКСТ2']
        if isinstance(text, str):
            lemmatized_text = lemmatize_text(text)
            df.at[i, 'ТЕКСТ3'] = lemmatized_text
            
      

    
def process_data_in_parallel(data, num_threads):               # распараллеливание процесса
    chunk_size = len(data) // num_threads
    chunks = [data.iloc[i:i+chunk_size] for i in range(0, len(data), chunk_size)]
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        executor.map(process_chunk, chunks)


print("Lemmatizing text...") 
process_data_in_parallel(df, 4)        # 4 - n - количество параллельных процессов
print(' ')
print("Successful!")
# сохранение измененного файла csv
df.to_csv('COMMENTS_KALIN_LEMMATIZED.csv', sep=';')
print("---------------------------------")

In [ ]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)
print("Calculating tonalities...")

dat_text = df['ТЕКСТ2']
results = model.predict(str(dat_text), k=3)

print("Successful!")
print("---------------------------------")

In [ ]:
df[['ПОЗИТИВНОСТЬ', 'НЕЙТРАЛЬНОСТЬ', 'НЕГАТИВНОСТЬ', 'СЕНТИМЕНТ', 'ТОНАЛЬНОСТЬ']] = 0
numofrows = df.shape[0] #Количество строк в файле
dat_text = list(df['ТЕКСТ2'])

dat_pos = df['ПОЗИТИВНОСТЬ']
dat_neu = df['НЕЙТРАЛЬНОСТЬ']
dat_neg = df['НЕГАТИВНОСТЬ'] 
print("Saving results of prediction...")

In [ ]:
def process_row(i):
    if type(dat_text[i]) is str:
        try:
            df.loc[i, 'ПОЗИТИВНОСТЬ'] = results[i]['positive']
        except KeyError:
            df.loc[i, 'ПОЗИТИВНОСТЬ'] = 0
        try:
            df.loc[i, 'НЕЙТРАЛЬНОСТЬ'] = results[i]['neutral']
        except KeyError:
            df.loc[i, 'НЕЙТРАЛЬНОСТЬ'] = 0
        try:
            df.loc[i, 'НЕГАТИВНОСТЬ'] = results[i]['negative']
        except KeyError:
            df.loc[i, 'НЕГАТИВНОСТЬ'] = 0
    else:
        df.loc[i, 'ПОЗИТИВНОСТЬ'] = 0
        df.loc[i, 'НЕЙТРАЛЬНОСТЬ'] = 0
        df.loc[i, 'НЕГАТИВНОСТЬ'] = 0

    if (dat_pos[i] > dat_neu[i]) and (dat_pos[i] > dat_neg[i]):
        df.loc[i, 'СЕНТИМЕНТ'] = 1
        df.loc[i, 'ТОНАЛЬНОСТЬ'] = dat_pos[i]
    elif (dat_neu[i] > dat_neg[i]):
        df.loc[i, 'СЕНТИМЕНТ'] = 2
        df.loc[i, 'ТОНАЛЬНОСТЬ'] = dat_neu[i]
    else:
        df.loc[i, 'СЕНТИМЕНТ'] = 3
        df.loc[i, 'ТОНАЛЬНОСТЬ'] = dat_neg[i]
        
        
with ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(process_row, range(numofrows))



print("\nSuccessful!")
print("---------------------------------")

In [ ]:
print(df.head())